# Data Integration

This notebook aims to show how each data type is integrated into CancerSig.

See https://gdc.cancer.gov/about-data/gdc-data-processing/biospecimen-data-standardization

## Slide Data Exploration

In [ ]:
from openslide import OpenSlide
import math
from PIL import Image

filename = "/mnt/Cancer_1/data/TCGA-EM-A3FO-01Z-00-DX1.84929069-A2FF-4588-98AF-77DE5D66F1C9.svs"

# Load the image
slide = OpenSlide(filename)

# Calculate the level of the image pyramid to read from
target_size = (640, 640)
width, height = slide.dimensions
level = slide.get_best_level_for_downsample(max(width, height) / max(target_size))
level -= 1
if level < 0:
    level = 0
    
level_width, level_height = slide.level_dimensions[level]

# Calculate the resize dimensions
target_size = (1000, 1000)
resize_width, resize_height = target_size

# Read the region at the desired level
region = slide.read_region((0, 0), level, (level_width, level_height))

# Resize the region to the target size
resized_region = region.resize((resize_width, resize_height), Image.BICUBIC)

region_rgb = resized_region.convert('RGB')

# Display the region
region_rgb.show()
slide.close()

In [ ]:
import pandas as pd
# miRNA

filename = f"/mnt/Cancer_1/data/{df_metadata.loc[(df_metadata['data.data_category'] == 'Transcriptome Profiling') &  (df_metadata['data.experimental_strategy'] == 'miRNA-Seq')]['data.file_name'].iloc[120]}"

df = pd.read_csv(filename, sep='\t')
print(df)


# Count them
miRNA_counts = {}
for filename in df_metadata.loc[(df_metadata['data.data_category'] == 'Transcriptome Profiling') &  (df_metadata['data.experimental_strategy'] == 'miRNA-Seq')]['data.file_name']:
    df = pd.read_csv(f"/mnt/Cancer_1/data/{filename}", sep='\t')
    
    for miRNA in df['miRNA_ID']:
        if miRNA in miRNA_counts:
            miRNA_counts[miRNA] += 1
        else:
            miRNA_counts[miRNA] = 1


## Metadata Loading

In [ ]:
import requests
import pandas as pd
from time import sleep

url = "https://api.gdc.cancer.gov/files"
params = {
    "fields": """file_id,demographic.cause_of_death,index_files.created_datetime,cases.days_to_index,annotations.notes,created_datetime,cases.demographic.created_datetime,cases.demographic.updated_datetime,cases.diagnoses.updated_datetime,cases.diagnoses.created_datetime,updated_datetime,cases.diagnoses.cog_liver_stage,cases.diagnoses.classification_of_tumor,cases.diagnoses.clark_level,cases.diagnoses.child_pugh_classification,cases.diagnoses.cancer_detection_method,cases.diagnoses.burkitt_lymphoma_clinical_variant,cases.diagnoses.best_overall_response,cases.diagnoses.ann_arbor_pathologic_stage,cases.diagnoses.ann_arbor_extranodal_involvement,cases.diagnoses.ann_arbor_clinical_stage,cases.diagnoses.ann_arbor_b_symptoms_described,cases.diagnoses.ann_arbor_b_symptoms,cases.diagnoses.ajcc_staging_system_edition,cases.diagnoses.ajcc_pathologic_t,cases.diagnoses.ajcc_pathologic_stage,cases.diagnoses.ajcc_pathologic_n,cases.diagnoses.ajcc_pathologic_m,cases.diagnoses.ajcc_clinical_t,cases.diagnoses.ajcc_clinical_stage,cases.diagnoses.ajcc_clinical_n,cases.diagnoses.ajcc_clinical_m,cases.diagnoses.adrenal_hormone,cases.diagnoses.tissue_or_organ_of_origin,cases.diagnoses.site_of_resection_or_biopsy,cases.diagnoses.primary_diagnosis,cases.diagnoses.morphology,cases.diagnoses.diagnosis_is_primary_disease,cases.diagnoses.age_at_diagnosis,cases.diagnosis,cases.demographic.race,cases.demographic.vital_status,cases.demographic.age_at_index,cases.demographic.age_is_obfuscated,cases.demographic.cause_of_death,cases.demographic.cause_of_death_source,cases.demographic.country_of_birth,cases.demographic.country_of_residence_at_enrollment,cases.demographic.days_to_birth,cases.demographic.days_to_death,cases.demographic.education_level,cases.demographic.marital_status,cases.demographic.occupation_duration_years,cases.demographic.premature_at_birth,cases.demographic.weeks_gestation_at_birth,cases.demographic.year_of_birth,cases.demographic.year_of_death,cases.demographic.ethnicity,cases.demographic.gender,cases.disease_type,cases.primary_site,cases.index_date,cases.consent_type,cases.days_to_consent,cases.days_to_lost_to_followup,cases.lost_to_followup,file_name,cases.submitter_id,cases.case_id,data_category,data_type,cases.samples.tumor_descriptor,cases.samples.tissue_type,cases.samples.sample_type,cases.samples.submitter_id,cases.samples.sample_id,analysis.workflow_type,cases.project.project_id,cases.samples.portions.analytes.aliquots.aliquot_id,cases.samples.portions.analytes.aliquots.submitter_id""",
    "from": 0,
    "size": 1000
}

all_data = pd.DataFrame()

while True:
    # make GET request with parameters
    response = requests.get(url, params=params)

    # check that the request was successful
    if response.status_code == 200:
        data = response.json()

        # check if there are data in the response
        if not data['data']['hits']:
            break

        # process data
        processed_data = []
        for hit in data['data']['hits']:
            row = hit.copy()

            # count the number of cases
            row['num_cases'] = len(row['cases'])

            # if there are any cases, take the first one
            if row['cases']:
                case = row['cases'][0].copy()
                row.update(case)

                # count the number of diagnoses
                row['num_diagnoses'] = len(case.get('diagnoses', []))

                # if there are any diagnoses, take the first two
                for i, diagnosis in enumerate(case.get('diagnoses', [])[:2]):
                    row.update({f'diagnosis_{i+1}_{k}': v for k, v in diagnosis.items()})

            processed_data.append(row)

        # add data to our dataframe
        df = pd.json_normalize(processed_data)
        all_data = pd.concat([all_data, df])

        # update 'from' for the next iteration
        params['from'] += params['size']
        print(params['from'])
    else:
        print("Request failed with status code", response.status_code)
        break


# saving to csv file
all_data.to_csv("all_data.csv", index=False)


In [ ]:
import pandas as pd
import os
filepaths = []
selected_ids = []

all_data = pd.read_csv('all_data.csv')

# This assumes data split between /mnt/Cancer_1 and /mnt/Cancer_2
for i, f in enumerate(all_data.file_name):
    if os.path.isfile(f"/mnt/Cancer_1/data/{f}"):
        filepaths.append(f"/mnt/Cancer_1/data/{f}")
        selected_ids.append(i)
    elif os.path.isfile(f"/mnt/Cancer_2/data/{f}"):
        filepaths.append(f"/mnt/Cancer_2/data/{f}")
        selected_ids.append(i)

processed_metadata = all_data.iloc[selected_ids]
processed_metadata['file_path'] = filepaths

# Dataloader

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
from openslide import OpenSlide
import math
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose, Resize, Normalize
import pandas as pd
import gzip
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
from transformers import BertTokenizer, BertModel

# TODO: Add methylation

from collections import Counter

# counter_start_pos = Counter()

for filepath in processed_metadata[processed_metadata['data_category'] == 'Simple Nucleotide Variation']['file_path']:
    num_comment_lines = sum(1 for line in gzip.open(filepath, 'rt') if line.startswith('#'))
    # maf the file, skipping the comment lines
    maf = pd.read_csv(filepath, low_memory=False, compression='gzip', sep='\t', skiprows=num_comment_lines)
    for sp in maf.ENSP:
        counter_start_pos[sp] += 1

top_100_start_positions = counter_start_pos.most_common(100)
top_100_start_positions = top_100_start_positions[1:]
top_100_ensp = top_100_start_positions

def parse_maf(filepath):
    num_comment_lines = sum(1 for line in gzip.open(filepath, 'rt') if line.startswith('#'))
    # Load the file, skipping the comment lines
    maf_df = pd.read_csv(filepath, low_memory=False, compression='gzip', sep='\t', skiprows=num_comment_lines)
    features_maf = ['gnomAD_AF',
                'gnomAD_AFR_AF',
                'gnomAD_AMR_AF',
                'gnomAD_ASJ_AF',
                'gnomAD_EAS_AF',
                'gnomAD_FIN_AF',
                'gnomAD_NFE_AF',
                'gnomAD_OTH_AF',
                'gnomAD_SAS_AF',
                'MAX_AF',
                'PolyPhen', # PolyPhen: benign(0.003)
                'SIFT',
#                 'BIOTYPE',
                'One_Consequence',
                'Consequence',
#                 'Feature_type',
#                 'Mutation_Status', All somatic
                'Variant_Classification',
                'Variant_Type',
                'ENSP'
               ]
    maf_df = maf_df[features_maf]
#     maf_df.PolyPhen = maf_df.PolyPhen.str.split('(')[1]
    top_100_ensp_ids = [t[0] for t in top_100_ensp]  # Extract the ENSP IDs

    # For each ENSP ID in the top 100 list
    for ensp in top_100_ensp_ids:
#         print( maf_df['ENSP'])
        # Add a new column to the maf_df DataFrame
        maf_df[f'ENSP_{ensp}'] = maf_df['ENSP'].apply(lambda x: x == ensp)
        
    for consequence in ['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant', 'intron_variant', 'splice_acceptor_variant', 'splice_donor_variant', 'splice_region_variant', 'non_coding_transcript_exon_variant', 'inframe_deletion', '3_prime_UTR_variant']:
        maf_df[f'one_consequence_{consequence}'] = maf_df['One_Consequence'].apply(lambda x: x == ensp)
        
    for consequence in ['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant', 'intron_variant', 'splice_acceptor_variant', 'splice_donor_variant', 'splice_region_variant', 'non_coding_transcript_exon_variant', 'inframe_deletion', '3_prime_UTR_variant']:
        maf_df[f'consequence_{consequence}'] = maf_df['Consequence'].apply(lambda x: x == ensp)
        
    for variant_class in ['Missense_Mutation', 'Silent', 'Nonsense_Mutation', 'Frame_Shift_Del', 'Splice_Site']:
        maf_df[f'variant_class_{consequence}'] = maf_df['Variant_Classification'].apply(lambda x: x == ensp)
    
    for variant_type in ['SNP', 'DEL', 'INS']:
        maf_df[f'variant_type_{consequence}'] = maf_df['Variant_Type'].apply(lambda x: x == ensp)
    
    maf_df["PolyPhen_score"] = maf_df["PolyPhen"].astype(str).str.extract("\((.*?)\)").astype(float)
    maf_df["SIFT_score"] = maf_df["SIFT"].astype(str).str.extract("\((.*?)\)").astype(float)
    maf_df.drop(columns=['PolyPhen', 'ENSP', 'SIFT', 'Variant_Type', 'Variant_Classification', 'Consequence', 'One_Consequence'], inplace=True)
    # Convert the DataFrame to a NumPy array
    numpy_array = maf_df.astype(float).values

    # Convert the NumPy array to a PyTorch tensor
    tensor = torch.from_numpy(numpy_array)
    tensor = torch.FloatTensor(numpy_array)
    return tensor


# filename = "/mnt/Cancer_1/data/TCGA-EM-A3FO-01Z-00-DX1.84929069-A2FF-4588-98AF-77DE5D66F1C9.svs"


full_miRNA = pd.read_csv(f"/mnt/Cancer_1/data/{processed_metadata.loc[(processed_metadata['data_category'] == 'Transcriptome Profiling') & (processed_metadata['data_type'] == 'miRNA Expression Quantification')]['file_name'].iloc[0]}", delimiter='\t', skiprows=0)[['miRNA_ID']]

# Proteomics Mapping
mapping_proteomics = pd.read_csv('mapping_proteomics.tsv', delimiter='\t').drop_duplicates(subset=['From'])
tcga_mapping = pd.read_csv('TCGA_antibodies_descriptions.gencode.v36.tsv', delimiter='\t').drop_duplicates()
tcga_mapping['gene_id'] = tcga_mapping['gene_id'].str.split('/')
tcga_mapping = tcga_mapping.explode('gene_id').drop_duplicates()
merged_proteomics = tcga_mapping.merge(mapping_proteomics, how='left', left_on='gene_id', right_on='From')


# pd.read_csv(f"/mnt/Cancer_1/data/{df_metadata.loc[(df_metadata['data.data_category'] == 'Transcriptome Profiling') & (df_metadata['data.data_type'] == 'Gene Expression Quantification')]['data.file_name'].iloc[210]}", delimiter='\t', skiprows=1)[['unstranded', 'stranded_first', 'stranded_second', 'tpm_unstranded', 'fpkm_unstranded', 'fpkm_uq_unstranded']]
def parse_gene_expression_quantification(filepath):
    df = pd.read_csv(f"{filepath}", delimiter='\t', skiprows=1)[['unstranded', 'stranded_first', 'stranded_second', 'tpm_unstranded', 'fpkm_unstranded', 'fpkm_uq_unstranded']]
    tensor = torch.FloatTensor(df.values)
    return tensor

# pd.read_csv(f"/mnt/Cancer_1/data/{df_metadata.loc[(df_metadata['data.data_category'] == 'Transcriptome Profiling') & (df_metadata['data.data_type'] == 'miRNA Expression Quantification')]['data.file_name'].iloc[123]}", delimiter='\t', skiprows=0)[['read_count', 'reads_per_million_miRNA_mapped']]
def parse_miRNA(filepath):
    df = pd.read_csv(f"{filepath}", delimiter='\t', skiprows=0)[['read_count', 'reads_per_million_miRNA_mapped']]
    tensor = torch.FloatTensor(df.values)
    return tensor

# df = pd.read_csv(f"/mnt/Cancer_1/data/{df_metadata.loc[(df_metadata['data.data_category'] == 'Transcriptome Profiling') & (df_metadata['data.data_type'] == 'Isoform Expression Quantification')]['data.file_name'].iloc[102]}", delimiter='\t', skiprows=0)[['miRNA_ID', 'read_count', 'reads_per_million_miRNA_mapped']].groupby('miRNA_ID').sum()
def parsed_isoform_to_miRNA(filepath):
    df = pd.read_csv(f"{filepath}", delimiter='\t', skiprows=0)[['miRNA_ID', 'read_count', 'reads_per_million_miRNA_mapped']].groupby('miRNA_ID').sum()
    full = full_miRNA.merge(df, how='left', on='miRNA_ID')[['read_count', 'reads_per_million_miRNA_mapped']]
    tensor = torch.FloatTensor(full.values)
    return tensor
    
def parse_svs(filepath):
    # Load the image
    slide = OpenSlide(filepath)

    # Calculate the level of the image pyramid to read from
    target_size = (2400, 2400)
    width, height = slide.dimensions
    level = slide.get_best_level_for_downsample(max(width, height) / max(target_size))
    level -= 1
    if level < 0:
        level = 0

    level_width, level_height = slide.level_dimensions[level]

    # Calculate the resize dimensions
    target_size = (2400, 2400)
    resize_width, resize_height = target_size

    # Read the region at the desired level
    region = slide.read_region((0, 0), level, (level_width, level_height))

    # Resize the region to the target size
    resized_region = region.resize((resize_width, resize_height), Image.BICUBIC)

    region_rgb = resized_region.convert('RGB')
    transform = Compose([
    ToTensor(),  # Convert the image to a tensor
        Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),  # Normalize the image with mean and standard deviation
    ])
    tensor_img = transform(region_rgb)
    return tensor_img



def parse_clinical(filepath):
    if filepath[-4:] == '.PDF':
        # Convert the PDF to images
        images = convert_from_path(f'{filepath}')

        # Initialize an OCR engine (pytesseract)
        pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'  # Update this path accordingly

        # Initialize a list to hold all the OCRed text
        text = []

        # Perform OCR on each image
        for i in range(len(images)):
            text.append(pytesseract.image_to_string(images[i]))

        # Concatenate all the text
        all_text = ' '.join(text)

    else:
        with open(f'{filepath}', 'r') as file:
            all_text = file.read()
    # Tokenize the text
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    inputs = tokenizer(all_text, truncation=True, return_tensors='pt', max_length=512)

    # Generate the embeddings
    model = BertModel.from_pretrained("bert-base-uncased")
    outputs = model(**inputs)

    # The last hidden state is the sequence of contextual embeddings. 
    embeddings = outputs.last_hidden_state[0, :, :].mean(0)
    return embeddings

def parse_copy_number_variation(filepath):
    # Copy number Segments
    df = pd.read_csv(f"{filepath}", sep='\t')
    df.loc[df.Chromosome == 'X', 'Chromosome'] = 23
    df.loc[df.Chromosome == 'Y', 'Chromosome'] = 24
    tensor = torch.from_numpy(df[['Chromosome', 'Start', 'End', 'Num_Probes', 'Segment_Mean']].astype(float).values)
    return tensor

def parse_gene_copy_number(filepath):
    # Gene copy number
    df = pd.read_csv(f"{filepath}", sep='\t')
    df['chromosome'] = df['chromosome'].str[3:]
    df.loc[df['chromosome'] == 'X', 'chromosome'] = 23
    df.loc[df['chromosome'] == 'Y', 'chromosome'] = 24
    if len(df) != 60623:
        print("ERROR")
    tensor = torch.from_numpy(df[['chromosome', 'start', 'end', 'copy_number', 'max_copy_number', 'min_copy_number']].astype(float).values)
    return tensor

def parse_allele_specific_copy_number(filepath):
    df = pd.read_csv(f"{filepath}", sep='\t')
    df['Chromosome'] = df['Chromosome'].str[3:]
    df.loc[df['Chromosome'] == 'X', 'Chromosome'] = 23
    df.loc[df['Chromosome'] == 'Y', 'Chromosome'] = 24
    tensor = torch.from_numpy(df[['Chromosome', 'Start', 'End', 'Copy_Number', 'Major_Copy_Number', 'Minor_Copy_Number']].astype(float).values)
    return tensor

def parse_proteomics(filepath):
    filename = f'{filepath}'
    df_proteome = pd.read_csv(filename, delimiter='\t').merge(merged_proteomics, how='right', on='AGID')
    tensor = torch.from_numpy(df_proteome['protein_expression'].values)
    
    return tensor # df_proteome <<< To merge

class MultinomialDataset(Dataset):
    def __init__(self, all_metadata):
        self.patients = all_metadata['case_id'].unique()
        self.metadata = all_metadata
        

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient = self.patients[idx]
        met = self.metadata[self.metadata['case_id'] == patient].sort_values('updated_datetime', ascending=False).drop_duplicates('data_type')
        
        files = met['file_path']
        
        data = {}
        
        # X parse_gene_expression_quantification
        # X parse_miRNA
        # X parsed_isoform_to_miRNA
        # X parse_svs
        # X parse_maf
        # X parse_clinical
        # X parse_copy_number_variation
        # X parse_gene_copy_number
        # X parse_allele_specific_copy_number
        # X parse_proteomics
        
        # TODO: Add methylation betas
        data['svs'] = torch.full((3, 2400, 2400), float('nan'))
        data['Masked Somatic Mutation'] = torch.full((1024, 135), float('nan')) # 437, 135
        data['Gene Level Copy Number'] = torch.full((60623, 6), float('nan'))
        data['Clinical'] = torch.full((768,), float('nan'))
        data['Copy Number Segment'] = torch.full((1024, 5), float('nan'))
        data['Protein Expression Quantification'] = torch.full((530,), float('nan'))
        data['miRNA Expression Quantification'] = torch.full((1881, 2), float('nan'))
        data['Isoform Expression Quantification'] = torch.full((1881, 2), float('nan'))
        data['Allele-specific Copy Number Segment'] = torch.full((1024, 6), float('nan'))
        data['Gene Expression Quantification'] = torch.full((60664, 6), float('nan'))
        
        for f in files:
            try:
                print(f)
                metadata = self.metadata[self.metadata.file_path == f].iloc[0]
                if f.endswith('.svs'):
                    data['svs'] = parse_svs(f)
                elif metadata.data_type in ['Masked Somatic Mutation', ]:
                    temp_somatic = parse_maf(f)[:1024, :]
                    data['Masked Somatic Mutation'][:temp_somatic.size()[0], :] = temp_somatic
                elif metadata.data_type in ['Gene Level Copy Number', ]:
                    data['Gene Level Copy Number'] = parse_gene_copy_number(f)
                elif metadata.data_type in ['Clinical Supplement', 'Pathology Report']:
                    data['Clinical'] = parse_clinical(f)
                elif metadata.data_type in ['Copy Number Segment', 'Masked Copy Number Segment']:
                    temp_copy = parse_copy_number_variation(f)
                    data['Copy Number Segment'][:temp_copy.size()[0], :] = temp_copy
                elif metadata.data_type in ['Protein Expression Quantification', ]:
                    temp_proteom = parse_proteomics(f)
                    data['Protein Expression Quantification'] = temp_proteom
                elif metadata.data_type in ['miRNA Expression Quantification', ]:
                    data['miRNA Expression Quantification'] = parse_miRNA(f)
                elif metadata.data_type in ['Isoform Expression Quantification', ]:
                    data['Isoform Expression Quantification'] = parsed_isoform_to_miRNA(f)
                elif metadata.data_type in ['Allele-specific Copy Number Segment', ]:
                    temp_allel = parse_allele_specific_copy_number(f)[:1024, :]
                    data['Allele-specific Copy Number Segment'][:temp_allel.size()[0], :] = temp_allel
                elif metadata.data_type in ['Gene Expression Quantification', ]:
                    data['Gene Expression Quantification'] = parse_gene_expression_quantification(f)
            except Exception as e:
                print(e)
        # Concatenate all the tensors together, fill in a constant value for missing files.
#         data = torch.cat([d for k, d in data.items() if d is not None], dim=0)
        return data

# # Example usage:

dataset = MultinomialDataset(processed_metadata)

dataloader = DataLoader(dataset, batch_size=3, shuffle=True)
for batch in dataloader:
    b = batch
    break
# dataset = MultiOmicsDataset(root_dir='/path/to/your/data')
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# for batch in dataloader:
#     # process your batch

# filename = "/mnt/Cancer_1/data/TCGA-EM-A3FO-01Z-00-DX1.84929069-A2FF-4588-98AF-77DE5D66F1C9.svs"
# a = parse_svs(filename).cuda()

## Pre-Processing of Data

In [ ]:

np.savetxt("/mnt/Cancer_2/processed_data/patients.csv", dataset.patients, delimiter=",", fmt='%s')
for i in range(0, len(dataset)):
    print(f"{i} / {len(dataset)}")
    data = dataset[i]
    for k, v in data.items():
        torch.save(v, f"/mnt/Cancer_2/processed_data/{i}_{'_'.join(k.split(' '))}.pt")

## Function to match swissprot to ensembl

In [ ]:
import requests

def get_swissprot_id(ensembl_protein_id):
    url = f"https://www.uniprot.org/uploadlists/"
    params = {
        "from": "ENSEMBL_ID",
        "to": "SWISSPROT",
        "format": "tab",
        "query": ensembl_protein_id,
    }

    response = requests.get(url, params=params)

    if not response.ok:
        response.raise_for_status()
        return None

    lines = response.text.strip().split("\n")
    if len(lines) < 2:
        return None

    # Parse the first line to get the column headers
    headers = lines[0].split("\t")

    # Parse the second line to get the data
    data = lines[1].split("\t")

    if len(data) < 2:
        return None

    # Assuming the Swiss-Prot ID is in the first column
    swissprot_id_with_version = data[0]

    return swissprot_id_with_version

# Example usage
ensembl_protein_id = "ENSP00000366934.13"
swissprot_id = get_swissprot_id(ensembl_protein_id)
print(swissprot_id)
